<a href="https://colab.research.google.com/github/sushant-97/keras_projects/blob/main/Using_pre_trained_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# Download dataset
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 5s 0us/step


In [3]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['sci.electronics', 'talk.politics.guns', 'comp.os.ms-windows.misc', 'sci.med', 'rec.sport.baseball', 'rec.sport.hockey', 'rec.autos', 'comp.sys.mac.hardware', 'rec.motorcycles', 'talk.religion.misc', 'talk.politics.mideast', 'comp.sys.ibm.pc.hardware', 'misc.forsale', 'soc.religion.christian', 'comp.graphics', 'alt.atheism', 'sci.crypt', 'talk.politics.misc', 'sci.space', 'comp.windows.x']
Number of files in comp.graphics: 1000
Some example filenames: ['38548', '39621', '38658', '38748', '38988']


In [4]:
# Here's a example of what one file contains:
print(open(data_dir / "comp.graphics" / "38987").read())

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



Header lines are leaking the category, either explicitly - first line is category name or implicitly - via organization filled.
Let's remove these

In [6]:
samples = []
labels = []
class_names = []
class_index = 0

for dirname in sorted(os.listdir(data_dir)):
  class_names.append(dirname)
  dirpath = data_dir/ dirname
  fnames = os.listdir(dirpath)
  print("Processing %s, %d files found" % (dirname, len(fnames)))

  for fname in fnames:
    fpath = dirpath / fname
    f = open(fpath, encoding = 'latin-1')
    content = f.read()
    lines = content.split("\n")
    lines = lines[10:]
    content = "\n".join(lines)
    samples.append(content)
    labels.append(class_index)
  class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [7]:
# This is a balanced classification problem

# Shuffle and split the data into training & validation sets

In [9]:
# Shuffle the data
seed = 1337

rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Training and validation split
validation_split = 0.2
num_vaidation_samples = int(validation_split * len(samples))
train_samples = samples[ : -num_vaidation_samples]
val_samples = samples[-num_vaidation_samples : ]

train_labels = labels[ : -num_vaidation_samples]
val_labels = labels[-num_vaidation_samples : ]

In [10]:
# Creating a vocabulary index

Let's use the TextVectorization to index the vocabulary found in the dataset. Later, we'll use the same layer instance to vectorize the samples.

Our layer will only consider the top 20,000 words, and will truncate or pad sequences to be actually 200 tokens long.

In [13]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens = 20000, output_sequence_length = 200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [19]:
# Retriving computed vocabulary used
vectorizer.get_vocabulary()[:10]

['', '[UNK]', 'the', 'to', 'of', 'a', 'and', 'in', 'is', 'i']

In [22]:
# Vectoring text sequence
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3554, 1793,   15,    2, 6340])

In [23]:
## 0 index - padding
## 1 index - out of vocabulary tockens

In [24]:
# dict mapping words to their indices:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [25]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3554, 1793, 15, 2, 6340]

# Load pre-trained word embeddings
Using GloVe Embeddings

In [26]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-10-21 14:33:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-10-21 14:33:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-10-21 14:33:41--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [28]:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "/content/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [29]:
# Embedding matrix that can be used in Keras Embedding layer
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # Words not found in embedding index will be all-zeros.
    # This includes the representation for "padding" and "OOV"
    embedding_matrix[i] = embedding_vector
    hits += 1
  else:
    misses += 1

print("Converted %d words (%d misses)" % (hits, misses))


Converted 18013 words (1987 misses)


*  Loading pre-trained word embeddings matrix into an Embedding layer
*   Keeping trainable = False, so as to keep the embeddings fixed



In [30]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer = keras.initializers.Constant(embedding_matrix),
    trainable = False,
)

# Build the model

In [31]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape = (None, ), dtype = 'int64')
embedded_sequences = embedding_layer(int_sequences_input)

x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000200   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 128)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

# Train the model

In [32]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [33]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 32s 13ms/step - loss: 2.6684 - acc: 0.1436 - val_loss: 2.0456 - val_acc: 0.2966
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 1.9373 - acc: 0.3312 - val_loss: 1.6134 - val_acc: 0.4424
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 1.5242 - acc: 0.4751 - val_loss: 1.3002 - val_acc: 0.5571
Epoch 4/20
125/125 [==============================] - 1s 11ms/step - loss: 1.2715 - acc: 0.5598 - val_loss: 1.1422 - val_acc: 0.6132
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 1.0930 - acc: 0.6189 - val_loss: 1.0985 - val_acc: 0.6239
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 0.9694 - acc: 0.6581 - val_loss: 1.0678 - val_acc: 0.6329
Epoch 7/20
125/125 [==============================] - 1s 11ms/step - loss: 0.8674 - acc: 0.6948 - val_loss: 1.0928 - val_acc: 0.6399
Epoch 8/20
125/125 [==============================] - 1s 9ms/step - loss

Let's export a Model object that takes as input a string of arbitrary length, rather than a sequence of indices.
It would make model much more portable, since we wouldn't have to worry about the input processing pipeline

In [35]:
string_input = keras.Input(shape = (1,), dtype = 'string')
x = vectorizer(string_input)
preds = model(x)

end2end_model = keras.Model(string_input, preds)

probabilities = end2end_model.predict([["this message is about computer graphics and 3D modeling"]])

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 265ms/step


'comp.graphics'